In [1]:
#TODO: Writeup with linking to code
#What is better to use after SVM? Poly or RBF Kernel?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from Vis import ClassificationVis, LabelVis
from Evals import *
from Data_Provider import *
from Util import ClassificationUtils as util
from Util import DataUtils as dutil

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

/home/emil/miniconda3/envs/emocog/lib/python3.7/site-packages/hypertools/plot/__init__.py:10: UserWarning: Could not switch backend to TkAgg.  This may impact performance of the plotting functions.
  warnings.warn('Could not switch backend to TkAgg.  This may impact performance of the plotting functions.')
/home/emil/miniconda3/envs/emocog/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
#important configs
sliding = 10
wsize = 100
s_sample = 0
e_sample = 35000
s_sample_ev = 35000
e_sample_ev = None
cutoff=.2

In [ ]:
data = DataProvider(draw=True)
#for wsize in [50,100,200]:
x,y = data.get_data(wsize=wsize,sliding=sliding,s_sample=s_sample,e_sample=e_sample,cutoff=cutoff)
x_ev,y_ev =  data.get_data(wsize=wsize,sliding=sliding,s_sample=s_sample_ev,e_sample=e_sample_ev, train= False, cutoff=cutoff)
dutil.save_data_to_file(x,y,x_ev,y_ev,wsize=wsize,sliding=sliding,s_sample=s_sample,e_sample=e_sample, s_sample_ev=s_sample_ev, e_sample_ev=e_sample_ev, cutoff=cutoff)


In [5]:
x,y,x_ev,y_ev = dutil.get_data_from_file(wsize=wsize,sliding=sliding,s_sample=s_sample,e_sample=e_sample, s_sample_ev=s_sample_ev, e_sample_ev=e_sample_ev,cutoff=cutoff)
print(x.shape)
print(y.shape)
print(x_ev.shape)
print(y_ev.shape)

(3200, 89)
(3200,)
(577, 89)
(577,)


In [6]:
print(np.unique(y,return_counts=True))
print(np.unique(y_ev,return_counts=True))

(array([0., 1.]), array([2297,  903]))
(array([0., 1.]), array([496,  81]))


In [5]:
# Run classifier with cross-validation and plot ROC curves - NEW APPROACH
cv = StratifiedKFold(n_splits=10,random_state=1)
#results_df = pd.DataFrame(columns=('C','Gamma','Degree','Threshold','F1 Score Tr','Precision Tr','Recall Tr','PR AUC','ROC AUC'))
results_df = pd.DataFrame(columns=('C','Gamma','Degree','PR AUC','ROC AUC'))
degree=np.random.choice(np.arange(10)[1:],100)
cs = np.random.uniform(0,4,100)
gammas = 2**(np.random.choice(18,100)-15.)
for idx,(deg,c,g) in enumerate(zip(degree,cs,gammas)):
    classifier = svm.SVC(C=c,kernel='poly', probability = True,gamma =g,degree=deg,random_state=5)
    #thresh = util.get_optimal_threshold(classifier, cv, x, y, go_after_pr=True) # get threshold using cv
    print('Get PR')
    auc_pr = util.get_auc_score(classifier, cv, x, y, go_after_pr=True)
    print('Get AUC')
    auc_roc =util.get_auc_score(classifier, cv, x, y, go_after_pr=False)
    print('Predict all')
    #y_pred,_ = util.fit_predict(classifier, .5, x, y) # using that threshold, get predictions and f1 score
    #f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    #prec_tr,recall_tr = get_precision_recall(y_pred,y)
    #results_df.loc[idx] = [c,g,deg,.5,f1_tr,prec_tr,recall_tr, auc_pr,auc_roc]
    results_df.loc[idx] = [c,g,deg, auc_pr,auc_roc]
    print('Gamma = %.6f, Dim = %d, C= %.2f, AUC PR %.3f, AUC ROC %.3f' %(g,deg,c,auc_pr,auc_roc))

Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 2.000000, Dim = 7, C= 2.48, AUC PR 0.247, AUC ROC 0.334
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 0.062500, Dim = 8, C= 1.64, AUC PR 0.228, AUC ROC 0.288
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred

pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 0.000122, Dim = 5, C= 1.25, AUC PR 0.344, AUC ROC 0.490
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 2.000000, Dim = 1, C= 2.46, AUC PR 0.291, AUC ROC 0.370
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit


pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 0.003906, Dim = 7, C= 2.91, AUC PR 0.239, AUC ROC 0.345
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Predict all
Gamma = 0.031250, Dim = 8, C= 3.47, AUC PR 0.228, AUC ROC 0.288
Get PR
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
Get AUC
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit
pred
Score
fit


KeyboardInterrupt: 

In [6]:
results_df

,C,Gamma,Degree,PR AUC,ROC AUC
0,2.483320,2.000000,7.0,0.247050,0.333864
1,1.637144,0.062500,8.0,0.227652,0.287767
2,3.678145,0.031250,4.0,0.217061,0.286323
3,0.358498,0.000061,9.0,0.292693,0.369867
4,3.845966,0.000977,2.0,0.201290,0.222868
5,1.342470,0.000122,3.0,0.300229,0.431927
6,0.166188,0.000977,9.0,0.359933,0.520927
7,2.869676,0.031250,4.0,0.217061,0.286323
8,1.250914,0.015625,8.0,0.227652,0.287767
9,2.951917,0.003906,6.0,0.222812,0.289850


In [ ]:
# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10,random_state=1)
results_df = pd.DataFrame(columns=('C','Gamma','Degree','Threshold','F1 Score Tr','Precision Tr','Recall Tr','F1 Score Ev','Precision Ev','Recall Ev'))
degree=np.random.choice(np.arange(10)[1:],100)
cs = np.random.uniform(0,4,100)
gammas = 2**(np.random.choice(18,100)-15.)
for idx,(deg,c,g) in enumerate(zip(degree,cs,gammas)):
    classifier = svm.SVC(C=c,kernel='poly', probability = True,gamma =g,degree=deg,random_state=5)
    thresh = util.get_optimal_threshold(classifier, cv, x, y) # get threshold using cv
    y_pred,y_pred_ev = util.fit_predict(classifier, thresh, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)
    results_df.loc[idx] = [c,g,deg,thresh,f1_tr,prec_tr,recall_tr,f1_ev,prec_ev,recall_ev]
    print('Gamma = %.6f, Dim = %d, C= %.2f F1 on train: %.4f, F1 on Ev: %.4f' %(g,deg,c,f1_tr,f1_ev))

In [ ]:
#bar_plot_svc(scores_tr,scores_ev,hypers,threshs) #with thresholds
ClassificationVis.bar_plot_svc(results_df['F1 Score Tr'],results_df['F1 Score Ev'],degree) #without thresholds

In [ ]:
best_choice = util.get_best_hyperparas_results(results_df) #what were the best hyperparas?
print(best_choice)
best_c,best_g,best_d,best_thr =best_choice[:4]

In [ ]:
#get the conf matrices
classifier = svm.SVC(C=best_c,kernel='poly', probability=True,degree=best_d,gamma=best_g,random_state=5)
y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev)
ClassificationVis.conf_mat(y_pred,y)
ClassificationVis.conf_mat(y_pred_ev,y_ev)
plt.show()
ClassificationVis.plot_roc(x,y,svm.SVC(C=best_c, kernel='poly', probability=True,degree=best_d,gamma = best_g,random_state=5), 'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))
ClassificationVis.plot_roc(x_ev,y_ev,svm.SVC(C=best_c, kernel='poly', probability=True,degree=best_d,gamma = best_g,random_state=5), 'SVC w/ Poly Kernel, degree= {:.2f}, C= {:.2f} Gamma = {:.2f}'.format(best_d,best_c, best_g))

In [ ]:
# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10,random_state=0)
scores_tr = []
scores_ev = []
threshs = []
results_df = pd.DataFrame(columns=('C','Gamma','Threshold','F1 Score Tr','Precision Tr','Recall Tr','F1 Score Ev','Precision Ev','Recall Ev'))
cs = np.random.uniform(0,5,100)
gammas = 2**(np.random.choice(18,100)-15.)
for idx,(g,c) in enumerate(zip(gammas,cs)):
    classifier = svm.SVC(C=c,kernel='rbf', probability = True,gamma = g, random_state=5)
    thresh = util.get_optimal_threshold(classifier, cv, x, y) # get threshold using cv
    y_pred,y_pred_ev = util.fit_predict(classifier,thresh, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)
    results_df.loc[idx] = [c,g,thresh,f1_tr,prec_tr,recall_tr,f1_ev,prec_ev,recall_ev]
    print('Gamma = %.6f, C= %2.2f, F1 on train: %.4f, F1 on Ev: %.4f' % (g,c,f1_tr,f1_ev))
    

In [ ]:
#ClassificationVis.plot_svc(scores_tr,scores_ev,cs,label='C')
#get best hyperpara
best_choice = util.get_best_hyperparas_results(results_df) #what were the best hyperparas?
print(best_choice)
best_c,best_g,best_thr =best_choice[:3]

In [ ]:
#get f1 scores on whole training set
classifier = svm.SVC(C=best_c, kernel='rbf', probability=True,gamma = best_g,random_state=5)
y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev)
get_conf_mat(y_pred,y)
get_conf_mat(y_pred_ev,y_ev)
plt.show()

In [ ]:
ClassificationVis.plot_roc(x,y,svm.SVC(C=best_c, kernel='rbf', probability=True,gamma = best_g,random_state=5), 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))
ClassificationVis.plot_roc(x_ev,y_ev,svm.SVC(C=best_c, kernel='rbf', probability=True,gamma = best_g,random_state=5), 'SVC, RBF Kernel, C ={:.2f}, Gamma = {:.5f}'.format(best_c,best_g))

In [ ]:
#first try random forest:
# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=10,random_state=0)
results_df = pd.DataFrame(columns=('Number Estimators','Max Depth','Max Features','Threshold','F1 Score Tr','Precision Tr','Recall Tr','F1 Score Ev','Precision Ev','Recall Ev'))
est = np.random.choice(np.arange(130)[5:],100)
max_d = np.random.choice(np.arange(60)[1:],100)
max_f = np.random.choice(np.arange(min(x.shape))[1:],100)

for idx,(c,d,f) in enumerate(zip(est,max_d,max_f)):
    classifier = RandomForestClassifier(n_estimators=c, max_depth=d, max_features=f,
                              random_state=0)
    thresh = util.get_optimal_threshold(classifier, cv, x, y) # get threshold using cv
    y_pred,y_pred_ev = util.fit_predict(classifier, thresh, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)
    results_df.loc[idx] = [c,d,f,thresh,f1_tr,prec_tr,recall_tr,f1_ev,prec_ev,recall_ev]
    print('Number Estimators= %d, Max Depth = %d, Max Feat = %d, F1 on train: %.4f, F1 on Ev: %.4f' % (c,d,f,f1_tr,f1_ev))
    


In [ ]:
#get best hyperpara
best_choice = util.get_best_hyperparas_results(results_df) #what were the best hyperparas?
print(best_choice)
best_n,best_d,best_f,best_thr =best_choice[:4].astype(int)
#get f1 scores on whole training set

classifier = RandomForestClassifier(n_estimators=int(best_n), max_depth=int(best_d), max_features=int(best_f),
                              random_state=0)
y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev)
get_conf_mat(y_pred,y)
get_conf_mat(y_pred_ev,y_ev)
plt.show()

In [ ]:
ClassificationVis.plot_roc(x,y,RandomForestClassifier(n_estimators=best_n, max_depth=best_d, max_features=best_f,
                              random_state=0), 'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))
ClassificationVis.plot_roc(x_ev,y_ev,RandomForestClassifier(n_estimators=best_n, max_depth=best_d, max_features=best_f,
                              random_state=0), 'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))